<a href="https://colab.research.google.com/github/Oleonn/DataMining/blob/Classification/CNN_Classification_automatique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#-----Imports et installations-----
#Pour se connecter au drive et acceder a ses fichiers
from google.colab import drive
drive.mount('/content/drive', force_remount=True)